In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *
# from test_0712 import Trainer

from train_0807 import Trainer

In [2]:
# saved_dir = 'saved/0712_SirajMPRA_seq_feature/0714_223244'
# saved_dir = 'saved/0712_SirajMPRA_seq_4/0711_173327'
saved_dir = 'saved/0712_SirajMPRA_seq_feature_train3_valid1/0716_062024'

In [3]:
config_path = f'{saved_dir}/config.yaml'
config = load_config(config_path)
config['train'] = False
config['save_dir'] = saved_dir
config['load_saved_model'] = True
config['saved_model_path'] = os.path.join(saved_dir, 'checkpoint.pth')


config['device'] = get_free_gpu_ids()[0]
trainer = Trainer(config)
trainer.test()
torch.cuda.empty_cache()

2024-08-07 05:20:45 - INFO - Start non-distributed training on rank 0, cuda:0.
2024-08-07 05:20:52 - INFO - load saved model from saved/0712_SirajMPRA_seq_feature_train3_valid1/0716_062024/checkpoint.pth


100%|██████████| 862/862 [01:49<00:00,  7.89it/s]


In [6]:
y_pred_list = np.load(f'{saved_dir}/test_pred.npy')
print(y_pred_list.shape)

(1765218,)


In [15]:
df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/SirajMPRA_total.csv')
df1

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T
...,...,...,...,...,...,...,...,...,...,...,...
585600,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C
585601,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.409758,0.556338,2.843295,1.495515,True,chrX,155233794,A,G
585602,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.894634,-0.072647,1.973661,0.794848,False,chrX,155233794,A,G
585603,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,NaN,0.312172,0.165634,-0.041383,0.444472,True,chrX,155234730,T,C


In [16]:
# merge pred with df

# for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
#     df1[f'{cell_type}_pred'] = y_pred_list[:, i]
# df1


df2 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/MPRA_exp/pretrained_based_models/data/Sei_Siraj_features_flatten.csv')
df2.insert(3, 'pred', y_pred_list)

for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
    df = df2[df2['cell_type'] == cell_type][['seq', 'pred']]
    df.columns = ['seq', f'{cell_type}_pred']
    df1 = pd.merge(df1, df, on='seq', how='outer')
df1

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,NaN,0.467988,-0.100597,-0.154240,-0.167743,False,chr19,9294737,G,A,-0.058640,0.090441,NaN,-0.001590
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,NaN,0.352003,0.154159,-0.016322,0.115593,True,chr19,9294737,G,A,-0.056274,0.070440,NaN,0.007599
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTACTCAGAATAAG...,NaN,1.355601,1.336478,1.141780,1.082136,True,chr6,71999351,C,T,0.405688,1.684659,NaN,1.516747
3,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTACTCAGAATAAG...,NaN,1.335333,1.191276,1.053293,1.094121,False,chr6,71999351,C,T,0.399756,1.580948,NaN,1.347186
4,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTAGCTGTGCGTGG...,NaN,1.263261,2.557821,0.223262,0.521439,False,chr9,2491922,T,C,0.405667,1.805280,NaN,0.833430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586136,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGGAGATGGAATCTC...,NaN,0.555848,1.493321,-0.113372,0.376846,True,chr11,119116770,T,C,0.090110,0.759395,NaN,0.499272
586137,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,NaN,NaN,0.291217,0.004240,NaN,True,chr6,153341374,C,T,NaN,-0.062451,NaN,NaN
586138,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,NaN,NaN,-0.085334,-0.272771,NaN,False,chr6,153341374,C,T,NaN,-0.076727,NaN,NaN
586139,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,NaN,2.067129,2.275694,1.982118,2.346065,True,chr2,181602082,C,T,1.079161,3.400488,NaN,1.900796


In [17]:
df1 = df1.dropna(subset=['HepG2', 'K562', 'A549', 'HCT116', 'SKNSH'], how='all')
df1

,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,NaN,0.467988,-0.100597,-0.154240,-0.167743,False,chr19,9294737,G,A,-0.058640,0.090441,NaN,-0.001590
1,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGACAAGAGTTGGCA...,NaN,0.352003,0.154159,-0.016322,0.115593,True,chr19,9294737,G,A,-0.056274,0.070440,NaN,0.007599
2,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTACTCAGAATAAG...,NaN,1.355601,1.336478,1.141780,1.082136,True,chr6,71999351,C,T,0.405688,1.684659,NaN,1.516747
3,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTACTCAGAATAAG...,NaN,1.335333,1.191276,1.053293,1.094121,False,chr6,71999351,C,T,0.399756,1.580948,NaN,1.347186
4,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTAGCTGTGCGTGG...,NaN,1.263261,2.557821,0.223262,0.521439,False,chr9,2491922,T,C,0.405667,1.805280,NaN,0.833430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586136,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGGAGATGGAATCTC...,NaN,0.555848,1.493321,-0.113372,0.376846,True,chr11,119116770,T,C,0.090110,0.759395,NaN,0.499272
586137,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,NaN,NaN,0.291217,0.004240,NaN,True,chr6,153341374,C,T,NaN,-0.062451,NaN,NaN
586138,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACAGAGTCTCG...,NaN,NaN,-0.085334,-0.272771,NaN,False,chr6,153341374,C,T,NaN,-0.076727,NaN,NaN
586139,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTGAGACGGAGTCTTG...,NaN,2.067129,2.275694,1.982118,2.346065,True,chr2,181602082,C,T,1.079161,3.400488,NaN,1.900796


In [48]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(df1), dtype=bool)
data_split_dict['chrom'] = df1['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['train'] = df1['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = df1['chr'].isin(['chr1'])
data_split_dict['test'] = df1['chr'].isin(['chr2'])
data_split_dict['designed'] = df1['chr'].isin(['chr0'])

In [75]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

# df1['diff_exp'] = (df1[cell_types].max(axis=1) - df1[cell_types].min(axis=1)) > 2
# data_split_dict['const'] = ~df1['diff_exp']
# data_split_dict['diff']  = df1['diff_exp']

# print(data_split_dict['const'].sum(), data_split_dict['diff'].sum())
# print((data_split_dict['test'] & data_split_dict['diff']).sum())

df1['mean'] = df1[cell_types].mean(axis=1)
df1['std'] = df1[cell_types].std(axis=1)

threshold = df1['std'].quantile(0.2)
data_split_dict['cts'] = (df1['std'] >= threshold)
data_split_dict['ubiq'] = (df1['std'] < threshold)

print(data_split_dict['cts'].sum(), data_split_dict['ubiq'].sum())

# for cell_type in cell_types:
#     # df1[f'{cell_type}_zscore'] = (df1[cell_type] - df1['mean']) / df1['std']
#     # df1[f'{cell_type}_cts'] = df1[f'{cell_type}_zscore'].abs() > 1
#     df1[f'{cell_type}_cts'] = (df1[cell_type] - df1['mean']).abs() > 1
#     df1[f'{cell_type}_ubiq'] = ~df1[f'{cell_type}_cts']
#     print(f"{cell_type}_cts num = {df1[f'{cell_type}_cts'].sum()}")

467501 116875


In [76]:
# for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
#     df = df2[df2['cell_type'] == cell_type]
#     r = pearson(df['exp'], df['pred'])
#     print(f'cell_type pearson r = {r:.3f}')

In [77]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'cts', 'ubiq']:
            condition = data_split_dict[data_split]
            df = df1[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}'])
            print(f'{cell_type:6} true vs {cell_type_pred:6} true, {data_split:5}, pearson r = {r:.3f}')
        
        # for data_split_2 in ['cts', 'ubiq']:
        #     condition = df1[f'{cell_type}_{data_split_2}']
        #     df = df1[condition]
        #     r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}'])
        #     print(f'{cell_type:6} true vs {cell_type_pred:6} true, {data_split_2:5}, pearson r = {r:.3f}')
        print()

HepG2  true vs HepG2  true, total, pearson r = 1.000
HepG2  true vs HepG2  true, cts  , pearson r = 1.000
HepG2  true vs HepG2  true, ubiq , pearson r = 1.000

HepG2  true vs K562   true, total, pearson r = 0.787
HepG2  true vs K562   true, cts  , pearson r = 0.774
HepG2  true vs K562   true, ubiq , pearson r = 0.953

HepG2  true vs A549   true, total, pearson r = 0.738
HepG2  true vs A549   true, cts  , pearson r = 0.727
HepG2  true vs A549   true, ubiq , pearson r = 0.900

HepG2  true vs HCT116 true, total, pearson r = 0.867
HepG2  true vs HCT116 true, cts  , pearson r = 0.859
HepG2  true vs HCT116 true, ubiq , pearson r = 0.966

K562   true vs HepG2  true, total, pearson r = 0.787
K562   true vs HepG2  true, cts  , pearson r = 0.774
K562   true vs HepG2  true, ubiq , pearson r = 0.953

K562   true vs K562   true, total, pearson r = 1.000
K562   true vs K562   true, cts  , pearson r = 1.000
K562   true vs K562   true, ubiq , pearson r = 1.000

K562   true vs A549   true, total, pears

In [78]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['test']:
            for data_split_2 in ['cts', 'ubiq']:
                condition = data_split_dict[data_split] & df1[f'{cell_type}_{data_split_2}']
                df = df1[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}'])
                print(f'{cell_type:6} true vs {cell_type_pred:6} true, {data_split:5}, {data_split_2:4}, pearson r = {r:.3f}')

HepG2  true vs HepG2  true, test , cts , pearson r = 1.000
HepG2  true vs HepG2  true, test , ubiq, pearson r = 1.000
HepG2  true vs K562   true, test , cts , pearson r = -0.011
HepG2  true vs K562   true, test , ubiq, pearson r = 0.821


HepG2  true vs A549   true, test , cts , pearson r = 0.276
HepG2  true vs A549   true, test , ubiq, pearson r = 0.767
HepG2  true vs HCT116 true, test , cts , pearson r = 0.378
HepG2  true vs HCT116 true, test , ubiq, pearson r = 0.882
K562   true vs HepG2  true, test , cts , pearson r = 0.383
K562   true vs HepG2  true, test , ubiq, pearson r = 0.817
K562   true vs K562   true, test , cts , pearson r = 1.000
K562   true vs K562   true, test , ubiq, pearson r = 1.000
K562   true vs A549   true, test , cts , pearson r = 0.487
K562   true vs A549   true, test , ubiq, pearson r = 0.726
K562   true vs HCT116 true, test , cts , pearson r = 0.537
K562   true vs HCT116 true, test , ubiq, pearson r = 0.835
A549   true vs HepG2  true, test , cts , pearson r = 0.469
A549   true vs HepG2  true, test , ubiq, pearson r = 0.725
A549   true vs K562   true, test , cts , pearson r = 0.354
A549   true vs K562   true, test , ubiq, pearson r = 0.724
A549   true vs A549   true, test , cts , pearson r = 1.0

In [39]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in [cell_type]:
        for data_split in ['test']:
            for data_split_2 in ['cts', 'ubiq']:
                condition = data_split_dict[data_split] & df1[f'{cell_type}_{data_split_2}']
                df = df1[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
                print(f'{cell_type:6} true vs {cell_type_pred:6} pred, {data_split:5}, {data_split_2:4}, pearson r = {r:.3f}')

HepG2  true vs HepG2  pred, test , cts , pearson r = 0.719
HepG2  true vs HepG2  pred, test , ubiq, pearson r = 0.825
K562   true vs K562   pred, test , cts , pearson r = 0.795
K562   true vs K562   pred, test , ubiq, pearson r = 0.814
A549   true vs A549   pred, test , cts , pearson r = 0.708
A549   true vs A549   pred, test , ubiq, pearson r = 0.756
HCT116 true vs HCT116 pred, test , cts , pearson r = 0.791
HCT116 true vs HCT116 pred, test , ubiq, pearson r = 0.811


In [10]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'train', 'valid', 'test', 'const', 'diff']:
            condition = data_split_dict[data_split]
            df = df1[condition]
            r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, pearson r = {r:.3f}')
        print()

HepG2 true vs HepG2 pred, total, pearson r = 0.879
HepG2 true vs HepG2 pred, train, pearson r = 0.896
HepG2 true vs HepG2 pred, valid, pearson r = 0.796
HepG2 true vs HepG2 pred, test, pearson r = 0.792
HepG2 true vs HepG2 pred, const, pearson r = 0.881
HepG2 true vs HepG2 pred, diff, pearson r = 0.765

HepG2 true vs K562 pred, total, pearson r = 0.792
HepG2 true vs K562 pred, train, pearson r = 0.809
HepG2 true vs K562 pred, valid, pearson r = 0.707
HepG2 true vs K562 pred, test, pearson r = 0.704
HepG2 true vs K562 pred, const, pearson r = 0.809
HepG2 true vs K562 pred, diff, pearson r = 0.478

HepG2 true vs A549 pred, total, pearson r = 0.774
HepG2 true vs A549 pred, train, pearson r = 0.792
HepG2 true vs A549 pred, valid, pearson r = 0.681
HepG2 true vs A549 pred, test, pearson r = 0.687
HepG2 true vs A549 pred, const, pearson r = 0.787
HepG2 true vs A549 pred, diff, pearson r = 0.530

HepG2 true vs HCT116 pred, total, pearson r = 0.836
HepG2 true vs HCT116 pred, train, pearson r =

In [11]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
    for cell_type_pred in cell_types:
        for data_split in ['total', 'train', 'valid', 'test']:
            for data_split_2 in ['total', 'const', 'diff']:
                condition = data_split_dict[data_split] & data_split_dict[data_split_2]
                df = df1[condition]
                r = pearson(df[f'{cell_type}'], df[f'{cell_type_pred}_pred'])
                print(f'{cell_type} true vs {cell_type_pred} pred, {data_split}, {data_split_2}, pearson r = {r:.3f}')
            print()

HepG2 true vs HepG2 pred, total, total, pearson r = 0.879
HepG2 true vs HepG2 pred, total, const, pearson r = 0.881
HepG2 true vs HepG2 pred, total, diff, pearson r = 0.765

HepG2 true vs HepG2 pred, train, total, pearson r = 0.896
HepG2 true vs HepG2 pred, train, const, pearson r = 0.896
HepG2 true vs HepG2 pred, train, diff, pearson r = 0.799

HepG2 true vs HepG2 pred, valid, total, pearson r = 0.796
HepG2 true vs HepG2 pred, valid, const, pearson r = 0.802
HepG2 true vs HepG2 pred, valid, diff, pearson r = 0.625

HepG2 true vs HepG2 pred, test, total, pearson r = 0.792
HepG2 true vs HepG2 pred, test, const, pearson r = 0.799
HepG2 true vs HepG2 pred, test, diff, pearson r = 0.626

HepG2 true vs K562 pred, total, total, pearson r = 0.792
HepG2 true vs K562 pred, total, const, pearson r = 0.809
HepG2 true vs K562 pred, total, diff, pearson r = 0.478

HepG2 true vs K562 pred, train, total, pearson r = 0.809
HepG2 true vs K562 pred, train, const, pearson r = 0.826
HepG2 true vs K562 pre

In [ ]:
# 选择差异表达的序列
# condition_1 = (df1['HepG2'] - df1['K562']) > 2
# condition_2 = (df1['HepG2'] - df1['K562']) < -2
condition_1 = (df1['HepG2'] - df1['K562']).abs() >= 2
condition_2 = (df1['HepG2'] - df1['K562']).abs() < 2
print(condition_1.sum(), condition_2.sum())

print('model only trained on HepG2')
print('only consider diff exp seqs')
for cell_type in ['HepG2', 'K562', 'SKNSH']:
    for data_split in ['train', 'valid', 'test', 'designed']:
        print(cell_type, data_split)
        condition_0 = data_split_condition_dict[data_split]
        condition = condition_0 & condition_1
        r = pearsonr(df1[condition][f'{cell_type}'], df1[condition][f'HepG2_pred'])[0]
        print(f'pearson r = {r:.3f}')
        condition = condition_0 & condition_2
        r = pearsonr(df1[condition][f'{cell_type}'], df1[condition][f'HepG2_pred'])[0]
        print(f'pearson r = {r:.3f}')
    print()

In [ ]:
df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/GosaiMPRA/GosaiMPRA_total.csv')
for i, cell_type in enumerate(['HepG2']):
    df1[f'{cell_type}_pred'] = y_pred_list[:, i]
df1

In [9]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(df1), dtype=bool)
data_split_dict['chrom'] = df1['chr'].isin([f'chr{i}' for i in range(1, 23)])
data_split_dict['train'] = df1['chr'].isin([f'chr{i}' for i in range(3, 23)])
data_split_dict['valid'] = df1['chr'].isin(['chr1'])
data_split_dict['test'] = df1['chr'].isin(['chr2'])
data_split_dict['designed'] = df1['chr'].isin(['chr0'])


cell_types = ['HepG2', 'K562']
df1['diff_exp'] = (df1[cell_types].max(axis=1) - df1[cell_types].min(axis=1)) > 1
data_split_dict['const'] = ~df1['diff_exp']
data_split_dict['diff']  = df1['diff_exp']

In [13]:
for cell_type in ['HepG2', 'K562']:
    for cell_type_pred in ['HepG2', 'K562']:
        for data_split in ['total', 'train', 'valid', 'test', 'designed', 'const', 'diff']:
            condition = data_split_dict[data_split]
            r = pearson(df1[condition][f'{cell_type}'], df1[condition][f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs HepG2 pred, total data, pearson r = 0.788
HepG2 true vs HepG2 pred, train data, pearson r = 0.898
HepG2 true vs HepG2 pred, valid data, pearson r = 0.810
HepG2 true vs HepG2 pred, test data, pearson r = 0.809
HepG2 true vs HepG2 pred, designed data, pearson r = 0.366
HepG2 true vs HepG2 pred, const data, pearson r = 0.884
HepG2 true vs HepG2 pred, diff data, pearson r = 0.622



KeyError: 'K562_pred'

In [18]:
# saved_dir = 'saved/0625_SirajMPRA_A549/0625_025532'

HepG2_pred = np.load(f'saved/0531_GosaiMPRA_baseline/0613_231345/y_pred_list.npy')
K562_pred = np.load(f'saved/0531_GosaiMPRA_baseline/0618_001655/y_pred_list.npy')
A549_pred = np.load(f'saved/0625_SirajMPRA_A549/0625_025532/y_pred_list.npy')
HCT116_pred = np.load(f'saved/0625_SirajMPRA_HCT116/0625_025546/y_pred_list.npy')
y_pred_list = np.concatenate([HepG2_pred, K562_pred, A549_pred, HCT116_pred], axis=1)
# print(y_pred_list.shape)

df1 = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/Siraj_Gosai_concat.csv')
for i, cell_type in enumerate(['HepG2', 'K562', 'A549', 'HCT116']):
    df1[f'{cell_type}_pred'] = y_pred_list[:, i]
df1

/tmp/ipykernel_694925/4012534825.py:10: DtypeWarning: Columns (6,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv('/home/hxcai/cell_type_specific_CRE/data/SirajMPRA/Siraj_Gosai_concat.csv')


,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,origin,HepG2_pred,K562_pred,A549_pred,HCT116_pred
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677.0,G,A,NaN,0.603010,0.979300,0.089610,0.228658
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677.0,G,A,NaN,0.523534,0.916531,0.029055,0.214567
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697.0,T,C,NaN,-0.025966,-0.583377,0.334277,-0.240863
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697.0,T,C,NaN,-0.049796,-0.607013,0.293812,-0.277427
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764.0,C,T,NaN,-0.114398,-0.093055,-0.052438,-0.148054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661768,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,NaN,0.591628,-0.315267,0.068697,NaN,NaN,chr0,NaN,NaN,NaN,control,1.488066,-0.080938,0.131021,-0.371440
661769,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,NaN,-0.359782,-0.134821,-0.022281,NaN,NaN,chr0,NaN,NaN,NaN,control,-0.775348,-0.264292,0.012704,-0.667063
661770,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,NaN,0.489900,0.646673,0.462366,NaN,NaN,chr0,NaN,NaN,NaN,control,0.356786,0.398112,0.177435,0.297358
661771,AGCCACAACAAAACAGCCAGTTAATGTGGAAACAAGGTCGACAACT...,NaN,-0.014411,-0.061676,0.239693,NaN,NaN,chr0,NaN,NaN,NaN,control,1.167894,0.079996,0.012424,0.225529


In [21]:
for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            r = pearson(df1[condition][f'{cell_type}'], df1[condition][f'{cell_type_pred}_pred'])
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
            print('pearson r', r)
        print()

HepG2 true vs HepG2 pred, train data, pearson r = 0.926
pearson r 0.9259957006812582
HepG2 true vs HepG2 pred, valid data, pearson r = 0.817
pearson r 0.8171515482957136
HepG2 true vs HepG2 pred, test data, pearson r = 0.815
pearson r 0.8148071042476853
HepG2 true vs HepG2 pred, designed data, pearson r = 0.745
pearson r 0.744512519022009

HepG2 true vs K562 pred, train data, pearson r = 0.759
pearson r 0.7587345782678894
HepG2 true vs K562 pred, valid data, pearson r = 0.690
pearson r 0.6903916498834725
HepG2 true vs K562 pred, test data, pearson r = 0.685
pearson r 0.6847371073960745
HepG2 true vs K562 pred, designed data, pearson r = -0.296
pearson r -0.2956059602354766

HepG2 true vs A549 pred, train data, pearson r = 0.632
pearson r 0.6322560229429839
HepG2 true vs A549 pred, valid data, pearson r = 0.583
pearson r 0.582653973730981
HepG2 true vs A549 pred, test data, pearson r = 0.574
pearson r 0.573519726677123
HepG2 true vs A549 pred, designed data, pearson r = 0.033
pearson r 

In [52]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            x = df1[condition][f'{cell_type}']

            cell_types_pred = [f'{ct}' for ct in cell_types if ct!= cell_type]
            y = df1[condition][cell_types_pred].mean(axis=1)
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_types_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs ['K562', 'A549', 'HCT116'] pred, train data, pearson r = 0.852
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, valid data, pearson r = 0.843
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, test data, pearson r = 0.836
HepG2 true vs ['K562', 'A549', 'HCT116'] pred, designed data, pearson r = -0.292

K562 true vs ['HepG2', 'A549', 'HCT116'] pred, train data, pearson r = 0.819
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, valid data, pearson r = 0.815
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, test data, pearson r = 0.804
K562 true vs ['HepG2', 'A549', 'HCT116'] pred, designed data, pearson r = -0.292

A549 true vs ['HepG2', 'K562', 'HCT116'] pred, train data, pearson r = 0.779
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, valid data, pearson r = 0.782
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, test data, pearson r = 0.762
0 (less than 10%) values are non-nan.
A549 true vs ['HepG2', 'K562', 'HCT116'] pred, designed data, pearson r = nan

HCT116 true vs ['HepG2', 'K56

In [24]:
cell_types = ['HepG2', 'K562', 'A549', 'HCT116']

for cell_type in cell_types:
        for data_split in ['train', 'valid', 'test', 'designed']:
            condition = data_split_condition_dict[data_split]
            x = df1[condition][f'{cell_type}']

            cell_types_pred = [f'{ct}_pred' for ct in cell_types if ct!= cell_type]
            y = df1[condition][cell_types_pred].mean(axis=1)
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_types_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()

HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, train data, pearson r = 0.810
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.727
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, test data, pearson r = 0.723
HepG2 true vs ['K562_pred', 'A549_pred', 'HCT116_pred'] pred, designed data, pearson r = -0.133

K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, train data, pearson r = 0.811
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.745
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, test data, pearson r = 0.740
K562 true vs ['HepG2_pred', 'A549_pred', 'HCT116_pred'] pred, designed data, pearson r = -0.147

A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, train data, pearson r = 0.762
A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, valid data, pearson r = 0.708
A549 true vs ['HepG2_pred', 'K562_pred', 'HCT116_pred'] pred, test data,

In [60]:
for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['test']:
            condition = data_split_condition_dict[data_split] & ((df1['HepG2'] - df1['K562']).abs() > 2)
            x = df1[condition][f'{cell_type}']
            y = df1[condition][f'{cell_type_pred}_pred']
            print(len(x))
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()


for cell_type in ['HepG2', 'K562', 'A549', 'HCT116']:
    for cell_type_pred in ['HepG2', 'K562', 'A549', 'HCT116']:
        for data_split in ['test']:
            condition = data_split_condition_dict[data_split] & ((df1['HepG2'] - df1['K562']).abs() <= 2)
            x = df1[condition][f'{cell_type}']
            y = df1[condition][f'{cell_type_pred}_pred']
            print(len(x))
            r = pearson(x, y)
            print(f'{cell_type} true vs {cell_type_pred} pred, {data_split} data, pearson r = {r:.3f}')
        print()


880
HepG2 true vs HepG2 pred, test data, pearson r = 0.670

880
HepG2 true vs K562 pred, test data, pearson r = 0.044

880
HepG2 true vs A549 pred, test data, pearson r = 0.166

880
HepG2 true vs HCT116 pred, test data, pearson r = 0.258

880
K562 true vs HepG2 pred, test data, pearson r = 0.133

880
K562 true vs K562 pred, test data, pearson r = 0.695

880
K562 true vs A549 pred, test data, pearson r = 0.386

880
K562 true vs HCT116 pred, test data, pearson r = 0.447

880
A549 true vs HepG2 pred, test data, pearson r = 0.484

880
A549 true vs K562 pred, test data, pearson r = 0.390

880
A549 true vs A549 pred, test data, pearson r = 0.585

880
A549 true vs HCT116 pred, test data, pearson r = 0.524

880
HCT116 true vs HepG2 pred, test data, pearson r = 0.549

880
HCT116 true vs K562 pred, test data, pearson r = 0.445

880
HCT116 true vs A549 pred, test data, pearson r = 0.578

880
HCT116 true vs HCT116 pred, test data, pearson r = 0.654

43603
HepG2 true vs HepG2 pred, test data, pears